## 📦 Paso 1: Instalación

In [ ]:
# Instalar BESTLIB desde GitHub (branch restore)
!pip uninstall -y BESTLIB 2>/dev/null || true
!pip install git+https://github.com/NahiaEscalante/bestlib.git@restore

# Verificar instalación
import BESTLIB
print(f"✅ BESTLIB v{BESTLIB.__version__} instalado correctamente")

## 📊 Paso 2: Preparar Datos

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Crear dataset tipo Iris
iris_data = pd.DataFrame({
    'sepal_length': np.concatenate([
        np.random.normal(5.0, 0.5, 50),
        np.random.normal(5.9, 0.5, 50),
        np.random.normal(6.5, 0.5, 50)
    ]),
    'sepal_width': np.concatenate([
        np.random.normal(3.4, 0.4, 50),
        np.random.normal(2.8, 0.3, 50),
        np.random.normal(3.0, 0.3, 50)
    ]),
    'petal_length': np.concatenate([
        np.random.normal(1.5, 0.2, 50),
        np.random.normal(4.3, 0.5, 50),
        np.random.normal(5.5, 0.5, 50)
    ]),
    'petal_width': np.concatenate([
        np.random.normal(0.2, 0.1, 50),
        np.random.normal(1.3, 0.2, 50),
        np.random.normal(2.0, 0.3, 50)
    ]),
    'species': ['setosa'] * 50 + ['versicolor'] * 50 + ['virginica'] * 50
})

print(f"✅ Dataset: {len(iris_data)} muestras, {len(iris_data.columns)} columnas")
iris_data.head()

## 🎯 Test 1: Scatter Plot Interactivo

**Instrucciones:**
1. Arrastra el mouse sobre el gráfico para seleccionar puntos
2. Observa los datos seleccionados en la consola

In [ ]:
from BESTLIB import MatrixLayout

# Activar debug
MatrixLayout.set_debug(True)

# Configurar scatter plot
MatrixLayout.map_scatter(
    'S',
    iris_data,
    x_col='sepal_length',
    y_col='sepal_width',
    category_col='species',
    interactive=True,
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

layout = MatrixLayout("SSS")

# Variable para guardar selección
selected_items = []

def on_select(payload):
    global selected_items
    selected_items = payload.get('items', [])
    print(f"\n{'='*60}")
    print(f"📊 {len(selected_items)} puntos seleccionados")
    if selected_items:
        print(f"Especies: {set(item.get('species') for item in selected_items)}")
    print(f"{'='*60}\n")

layout.on('select', on_select)
layout

### Verificar Selección

In [ ]:
if selected_items:
    df = pd.DataFrame(selected_items)
    print(f"✅ {len(df)} elementos seleccionados")
    print("\nDistribución de especies:")
    print(df['species'].value_counts())
else:
    print("⚠️ No hay selección. Selecciona puntos en el scatter plot.")

## 🔗 Test 2: Linked Views

**Instrucciones:**
1. Selecciona puntos en el SCATTER PLOT (superior)
2. Observa cómo el BAR CHART e HISTOGRAM se actualizan automáticamente

In [ ]:
from BESTLIB import ReactiveMatrixLayout, SelectionModel

# Crear modelo de selección
selection_model = SelectionModel()

# Crear layout con vistas enlazadas
layout = ReactiveMatrixLayout(
    """
    SSS
    BBH
    """,
    selection_model=selection_model
)

layout.set_data(iris_data)

# 1. Scatter plot (vista principal)
layout.add_scatter(
    'S',
    x_col='petal_length',
    y_col='petal_width',
    category_col='species',
    interactive=True,
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

# 2. Bar chart enlazado
layout.add_barchart(
    'B',
    category_col='species',
    linked_to='S',  # 🔥 Enlazado al scatter
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

# 3. Histogram enlazado
layout.add_histogram(
    'H',
    column='sepal_length',
    bins=15,
    linked_to='S',  # 🔥 Enlazado al scatter
    axes=True,
    color='#9b59b6'
)

print("🔗 Linked Views: Selecciona en el scatter para ver actualizaciones")
layout.display()

### Verificar Datos Seleccionados

In [ ]:
count = selection_model.get_count()
items = selection_model.get_items()

print(f"📊 Elementos seleccionados: {count}")

if items:
    df = pd.DataFrame(items)
    print("\n📈 Distribución:")
    print(df['species'].value_counts())
    print("\n🔢 Estadísticas:")
    print(df[['petal_length', 'petal_width', 'sepal_length']].describe())
else:
    print("⚠️ No hay selección")

## 📊 Test 3: Bar Chart Interactivo

**Instrucciones:**
1. Haz CLICK en una barra
2. Los datos se guardarán en `bar_selected_data`

In [ ]:
from BESTLIB import ReactiveMatrixLayout, SelectionModel

bar_selection = SelectionModel()
bar_selected_data = []

layout_bar = ReactiveMatrixLayout("BBB", selection_model=bar_selection)
layout_bar.set_data(iris_data)

# Bar chart interactivo (vista principal)
layout_bar.add_barchart(
    'B',
    category_col='species',
    interactive=True,  # 🔥 Vista principal
    selection_var='bar_selected_data',
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

print("📊 Bar Chart Interactivo: Click en una barra")
layout_bar.display()

### Verificar Selección del Bar Chart

In [ ]:
if isinstance(bar_selected_data, pd.DataFrame) and not bar_selected_data.empty:
    print(f"✅ {len(bar_selected_data)} filas seleccionadas")
    print(bar_selected_data.head())
elif bar_selected_data:
    print(f"✅ {len(bar_selected_data)} elementos")
else:
    print("⚠️ No hay selección. Click en una barra.")

## 🎨 Test 4: Dashboard Completo

Dashboard con 5 visualizaciones, 4 sincronizadas

In [ ]:
from BESTLIB import ReactiveMatrixLayout, SelectionModel

dashboard_selection = SelectionModel()

dashboard = ReactiveMatrixLayout(
    """
    SSSBBB
    SSSHHH
    PPPCCC
    """,
    selection_model=dashboard_selection,
    gap=12
)

dashboard.set_data(iris_data)

# Scatter plot principal
dashboard.add_scatter(
    'S',
    x_col='petal_length',
    y_col='petal_width',
    category_col='species',
    interactive=True,
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

# Bar chart enlazado
dashboard.add_barchart(
    'B',
    category_col='species',
    linked_to='S',
    axes=True,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

# Histogram enlazado
dashboard.add_histogram(
    'H',
    column='sepal_length',
    bins=12,
    linked_to='S',
    axes=True,
    color='#9b59b6'
)

# Pie chart enlazado
dashboard.add_pie(
    'P',
    category_col='species',
    linked_to='S',
    donut=True,
    innerRadius=40,
    colorMap={
        'setosa': '#e74c3c',
        'versicolor': '#3498db',
        'virginica': '#2ecc71'
    }
)

# Correlation heatmap
dashboard.add_correlation_heatmap('C', showValues=False)

print("🎨 Dashboard Completo: Selecciona en el scatter para ver sincronización")
dashboard.display()

## 🔍 Test 5: Verificar Sistema de Comunicación

In [ ]:
status = MatrixLayout.get_status()

print("🔍 Estado del Sistema")
print("=" * 60)
print(f"✅ Comm registrado: {status['comm_registered']}")
print(f"🐛 Modo debug: {status['debug_mode']}")
print(f"📊 Instancias activas: {status['active_instances']}")
print(f"\n📋 IDs de instancias: {status['instance_ids']}")
print(f"\n🌐 Handlers globales: {status['global_handlers']}")
print("=" * 60)

## 📚 Test 6: Historial de Selecciones

In [ ]:
history = selection_model.get_history()

print(f"📚 Historial: {len(history)} selecciones")
print("=" * 60)

if history:
    for i, sel in enumerate(history[-5:], 1):
        print(f"\n{i}. {sel['timestamp']}")
        print(f"   Elementos: {sel['count']}")
else:
    print("⚠️ No hay historial. Selecciona puntos primero.")

## ✅ Resumen Final

In [ ]:
print("=" * 70)
print(" " * 20 + "✅ RESUMEN DE PRUEBAS")
print("=" * 70)

# Verificaciones
try:
    import BESTLIB
    print(f"✅ 1. Instalación: OK (v{BESTLIB.__version__})")
except:
    print("❌ 1. Instalación: FALLO")

try:
    assert len(iris_data) == 150
    print(f"✅ 2. Datos: OK ({len(iris_data)} muestras)")
except:
    print("❌ 2. Datos: FALLO")

try:
    print(f"✅ 3. Scatter Interactivo: OK ({len(selected_items)} seleccionados)")
except:
    print("❌ 3. Scatter Interactivo: FALLO")

try:
    count = selection_model.get_count()
    print(f"✅ 4. Linked Views: OK ({count} elementos)")
except:
    print("❌ 4. Linked Views: FALLO")

try:
    count_bar = bar_selection.get_count()
    print(f"✅ 5. Bar Chart Interactivo: OK ({count_bar} elementos)")
except:
    print("❌ 5. Bar Chart Interactivo: FALLO")

try:
    count_dash = dashboard_selection.get_count()
    print(f"✅ 6. Dashboard: OK ({count_dash} elementos)")
except:
    print("❌ 6. Dashboard: FALLO")

try:
    status = MatrixLayout.get_status()
    assert status['comm_registered']
    print(f"✅ 7. Sistema de Comunicación: OK")
except:
    print("❌ 7. Sistema de Comunicación: FALLO")

print("=" * 70)
print("\n🎉 ¡Pruebas completadas!")
print("\n💡 Nota: Si algunos tests muestran 0 elementos,")
print("   es porque aún no has hecho selecciones.")
print("   Regresa a las celdas anteriores y selecciona puntos.")
print("=" * 70)